# Read-out Error


In [6]:
# import packages
import numpy as np

import qiskit_aer.noise as noise
from qiskit_aer.noise.errors import coherent_unitary_error, amplitude_damping_error, ReadoutError
from qiskit.circuit.library import RXGate

from qiskit_aer.primitives import SamplerV2 as Sampler
from qiskit.circuit import QuantumCircuit
from qiskit.circuit.random import random_circuit
from qiskit.quantum_info import SparsePauliOp

In [75]:
def get_readout_error_matrix(X):
    """
    args:
        X: np.darray (Nx2) probability of false result from most to least significant bit
    returns:
        output: np.darray (2^N x 2^N) matrix of readout errors
    """
    # cast to np array
    try:
        X = np.array(X)
    except Exception as exception:
        print("Must pass in valid datatype: array of numbers.")
        print(exception)
    
    # check array dimensions
    D = X.shape[1]
    if D != 2:
        raise ValueError("Input probabilities must be in form [[probability of error when prepared 0, probability of error when prepared 1], ...]")

    # number data points
    N = X.shape[0]

    output = np.ones((1, 1))

    SIGN = np.array([[-1, 1], [1, -1]])

    # goal shape: 2^N x 2^N
    for i in range(N):
        # obtain confusion matrix
        p_i = X[i].reshape((2, 1)) # [[FN], [FP]]
        p_i = p_i * SIGN # [[-FN, FN], [FP, -FP]]
        p_i = p_i + np.eye(2) # [[TN, FN], [FP, TP]]

        # reshape matrices
        p_i_flattened = p_i.reshape((1, p_i.size))

        # recursively calculate dimensions
        output_new = np.ones((2 * output.shape[0], 2 * output.shape[0])) 

        # i
        for j in range(output.shape[0]):
            row = output[j].reshape((output[j].size, 1))

            prod = row * p_i_flattened # shape output[j].size x 4
            output_new[(2 * j)] = prod[:, 0:2].flatten()
            output_new[(2 * j + 1)] = prod[:, 2:4].flatten()

        output = output_new

    return output

In [ ]:
probabilities = [
    [0.01, 0.03]
]
get_readout_error_matrix(probabilities)

In [ ]:
probabilities = [
    [0.01, 0.03],
    [0.05, 0.02]
]
get_readout_error_matrix(probabilities)

array([[0.99, 0.01],
       [0.03, 0.97]])

In [67]:
def string_to_int_array(X):
    return np.array(list(X)).astype("i")

In [64]:
def get_probabilities_of_one(data):
	"""
	args:
		data (dict) dictionary of {"outcome": counts}

	returns:
		probabilities (1xN array) the value at index i is the probability of measuring a 1
	"""

	N = len(list(data.keys())[0])
	
	total_counts = np.zeros(N)

	for key in list(data.keys()):
		counts = data[key]

		arr = string_to_int_array(key)
		
		total_counts = total_counts + (counts * arr)
	
	nshots = np.sum(list(data.values()))

	probabilities = total_counts / nshots

	return probabilities

## Basic Readout Error

We consider a simple case of read-out error mitigation.

In [51]:
from qiskit_ibm_runtime.fake_provider import FakePerth
from qiskit_aer import AerSimulator
from qiskit import transpile
from qiskit_aer.noise import NoiseModel

backend = AerSimulator.from_backend(FakePerth())

In [44]:
N = 7

zeros = QuantumCircuit(N)
zeros.measure_all()

ones = QuantumCircuit(N)
ones.x(list(range(N)))
ones.measure_all()

In [45]:
zeros.draw()

░ ┌─┐                  
   q_0: ─░─┤M├──────────────────
         ░ └╥┘┌─┐               
   q_1: ─░──╫─┤M├───────────────
         ░  ║ └╥┘┌─┐            
   q_2: ─░──╫──╫─┤M├────────────
         ░  ║  ║ └╥┘┌─┐         
   q_3: ─░──╫──╫──╫─┤M├─────────
         ░  ║  ║  ║ └╥┘┌─┐      
   q_4: ─░──╫──╫──╫──╫─┤M├──────
         ░  ║  ║  ║  ║ └╥┘┌─┐   
   q_5: ─░──╫──╫──╫──╫──╫─┤M├───
         ░  ║  ║  ║  ║  ║ └╥┘┌─┐
   q_6: ─░──╫──╫──╫──╫──╫──╫─┤M├
         ░  ║  ║  ║  ║  ║  ║ └╥┘
meas: 7/════╩══╩══╩══╩══╩══╩══╩═
            0  1  2  3  4  5  6

In [46]:
ones.draw()

┌───┐ ░ ┌─┐                  
   q_0: ┤ X ├─░─┤M├──────────────────
        ├───┤ ░ └╥┘┌─┐               
   q_1: ┤ X ├─░──╫─┤M├───────────────
        ├───┤ ░  ║ └╥┘┌─┐            
   q_2: ┤ X ├─░──╫──╫─┤M├────────────
        ├───┤ ░  ║  ║ └╥┘┌─┐         
   q_3: ┤ X ├─░──╫──╫──╫─┤M├─────────
        ├───┤ ░  ║  ║  ║ └╥┘┌─┐      
   q_4: ┤ X ├─░──╫──╫──╫──╫─┤M├──────
        ├───┤ ░  ║  ║  ║  ║ └╥┘┌─┐   
   q_5: ┤ X ├─░──╫──╫──╫──╫──╫─┤M├───
        ├───┤ ░  ║  ║  ║  ║  ║ └╥┘┌─┐
   q_6: ┤ X ├─░──╫──╫──╫──╫──╫──╫─┤M├
        └───┘ ░  ║  ║  ║  ║  ║  ║ └╥┘
meas: 7/═════════╩══╩══╩══╩══╩══╩══╩═
                 0  1  2  3  4  5  6

In [57]:
# run the circuit on the simulator
backend = FakePerth()
noise_model = NoiseModel.from_backend(backend)
sampler = Sampler(options={
    'backend_options': {
        'noise_model': noise_model
	}
})
nshots = 1024

zeros_transpiled = transpile(zeros, backend)
zeros_job = sampler.run([zeros_transpiled], shots=nshots)
zeros_pub_result = zeros_job.result()[0]

zeros_counts = zeros_pub_result.data.meas.get_counts()


ones_transpiled = transpile(ones, backend)
ones_job = sampler.run([ones_transpiled], shots=nshots)
ones_pub_result = ones_job.result()[0]

ones_counts = ones_pub_result.data.meas.get_counts()

In [72]:
ones_false_probs = 1 - get_probabilities_of_one(ones_counts)
zeros_false_probs = get_probabilities_of_one(zeros_counts)

In [76]:
# calculate the read-out matrix
zipped = np.array([zeros_false_probs, ones_false_probs]).T
get_readout_error_matrix(zipped)

array([[8.01249835e-01, 1.35265613e-02, 1.75923118e-02, ...,
        8.96866809e-10, 1.16644284e-09, 1.96916864e-11],
       [1.90963218e-02, 7.95680075e-01, 4.19280518e-04, ...,
        5.27568711e-08, 2.78000279e-11, 1.15833450e-09],
       [2.47891666e-02, 4.18486427e-04, 7.94052981e-01, ...,
        2.77473763e-11, 5.26489881e-08, 8.88811119e-10],
       ...,
       [9.04155772e-10, 3.76731572e-08, 1.98517235e-11, ...,
        8.00339597e-01, 4.21735836e-04, 1.75723265e-02],
       [1.17369556e-09, 1.98141256e-11, 3.75961189e-08, ...,
        4.20937094e-04, 7.98702975e-01, 1.34835656e-02],
       [2.79728832e-11, 1.16553680e-09, 8.96034612e-10, ...,
        2.47610055e-02, 1.90356220e-02, 7.93150918e-01]], shape=(128, 128))